In [2]:
%%time
import bz2
import h5py
import numpy as np
import pandas as pd
import scipy.sparse as sps
from scipy.sparse import csr_matrix
import math
cache_dir="G:/datasets/DEE2/"
cache_dir="F:/test_data/DEE2/"
organism_name= 'ecoli'#'hsapiens' 'ecoli' mmusculus
geo_acc_name = 'GSE_accession' # may be "GSE_accession" or "Sample_name"
tx_info_file = cache_dir+"TxInfo/"+ organism_name+"_TxInfo.tsv"
metadata_file = cache_dir+"metadata/"+ organism_name+"_metadata.tsv"

tx_info=pd.read_csv(filepath_or_buffer=tx_info_file,sep="\t")
tx_to_gene= dict(zip(tx_info["TxID"],tx_info["GeneID"]))
tx_ind= {tx:ind for ind,tx in enumerate(tx_to_gene.keys())}
gene_ind= {gene:ind for ind,gene in enumerate(pd.unique(tx_info["GeneID"]))}
tx_to_gene_matrix= sps.lil_matrix((len(tx_ind),len(gene_ind)))
for tx_key in tx_to_gene:
    tx_to_gene_matrix[tx_ind[tx_key],gene_ind[tx_to_gene[tx_key]]]=1
print(len(gene_ind))
print(len(tx_ind))
print(len(tx_to_gene))
#meta_df=pd.read_csv(filepath_or_buffer=metadata_file,sep="\t")   
iter_meta = pd.read_csv(filepath_or_buffer=metadata_file,sep="\t", iterator=True, chunksize=1000)

#meta_df = pd.concat([chunk[chunk[geo_acc_name].str.startswith('GSM')] for chunk in iter_meta])
meta_df = pd.concat([chunk for chunk in iter_meta])
true_srr=len(meta_df)
print(true_srr)
meta_df=meta_df[meta_df[geo_acc_name].str.startswith('GSM')]
n_srr= len(meta_df)
print(n_srr)
srr_to_gsm= dict(zip(meta_df["SRR_accession"],meta_df[geo_acc_name]))
srr_ind= {srr:ind for ind,srr in enumerate(srr_to_gsm.keys())}
gsm_ind= {gsm:ind for ind,gsm in enumerate(pd.unique(meta_df[geo_acc_name]))}
gsm_from_srr_matrix= sps.lil_matrix((len(gsm_ind),len(srr_ind)))
for srr_key in srr_to_gsm:
    gsm_from_srr_matrix[gsm_ind[srr_to_gsm[srr_key]],srr_ind[srr_key]]=1


4322
4322
4322
7024
3596
Wall time: 382 ms


In [8]:
%%time
import datetime
import dask.dataframe as dd
from scipy.sparse import isspmatrix_csr,csr_matrix,csc_matrix,issparse,isspmatrix_csr
data_file_name= cache_dir+"raw_files/"+organism_name+"_ke.tsv.bz2"
#data_file_name="F://mmusculus_ke.tsv.bz2"
h5_gsm_name = cache_dir+organism_name+ "_kalisto_matrix.h5"
now = datetime.datetime.now()
print("start h5")
with h5py.File(h5_gsm_name, 'w') as h5_gse:
    meta_grp = h5_gse.create_group('meta')
    info_grp = h5_gse.create_group('info')
    data_grp= h5_gse.create_group('data')
    info_grp.create_dataset('version', data="dee2_gse_v1")
    info_grp.create_dataset('creation_date', data=now.strftime("%Y.%m.%d"))
    n_genes =len(gene_ind)
    dt = h5py.special_dtype(vlen=np.str)
    meta_grp.create_dataset('genes',data=gene_ind.keys(),dtype=dt)
    n_gsm=len(gsm_ind)        
    meta_grp.create_dataset('Sample_geo_accession',data=gsm_ind.keys(), dtype =dt)
    meta_grp.create_dataset('Sample_instrument_model',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_last_update_date',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_library_selection',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_library_source',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_library_strategy',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_organism_ch1',(n_gsm,),dtype =dt)
    meta_grp.create_dataset('Sample_series_id',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_status',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_submission_date',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_taxid_ch1',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_quality',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_title',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_type',data=np.full(n_gsm,"SRA", dtype =dt))
    exp_data=data_grp.create_dataset("expression", (n_gsm,n_genes),dtype= 'i4')#, compression="gzip", compression_opts=9)
    import dask.array as da
    dask_exp = da.from_array(exp_data, chunks=(1000,1000))
    processed_gse=0
    srr_per_time=1000
    row_length= len(tx_ind)
    print(row_length)
    df_data = pd.read_csv(filepath_or_buffer=data_file_name,sep="\t", names=["srr","tx","ke"])
    df_data
print(1)

start h5
4322
1
Wall time: 25.8 s
